In [28]:
import MySQLdb
import pandas as pd

In [29]:
def central_server_frequencies():
    
    # Make a mysql connection
    mysql_cn= MySQLdb.connect(host='127.0.0.1', port=5003,user='root', passwd='cc3721b', db='edc')

    # Visit per survey
    sql_statement = "SELECT * from bcpp_subject_subjectvisit INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_visit_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['visit_code'])    
    visits = subject_visit_df.groupby('visit_code').size()
    
    # Consents
    sql_statement = "SELECT * from bcpp_subject_subjectconsent INNER JOIN member_householdmember ON bcpp_subject_subjectconsent.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_consent_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    consents = subject_consent_df.groupby('map_area').size()
    
    # Household Members
    sql_statement = "SELECT * from member_householdmember INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    household_member_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    members = household_member_df.groupby('map_area').size()
    
    # Requisitions
    sql_statement = "SELECT * FROM bcpp_subject_subjectrequisition INNER JOIN bcpp_subject_subjectvisit ON bcpp_subject_subjectrequisition.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
    subject_requiaitions_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['panel_name', 'visit_code'])
    requisitions = subject_requiaitions_df.groupby(['visit_code', 'panel_name']).size()
    
    data_dict = {
        'Subject Visits': visits.to_dict(),
        'Subject Consents': consents.to_dict(),
        'Household Members': members.to_dict(),
        'Subject requisitions': requisitions.to_dict(),
    }

    crf_list = [
        'demographics', 'education', 'hivtestinghistory', 'hivtestreview', 'residencymobility',
        'hivresultdocumentation', 'hivtested', 'hivcareadherence', 'sexualbehaviour', 'recentpartner',
        'secondpartner', 'thirdpartner', 'hivmedicalcare', 'accesstocare', 'circumcision', 'circumcised',
        'uncircumcised', 'reproductivehealth', 'medicaldiagnoses', 'hypertensioncardiovascular', 'heartattack',
        'cancer', 'hivrelatedillness', 'tuberculosis', 'tbsymptoms', 'qualityoflife', 'resourceutilization',
        'outpatientcare', 'hospitaladmission', 'hivhealthcarecosts', 'labourmarketwages',
        'hivlinkagetocare', 'hivresult', 'elisahivresult', 'pimacd4', 'hicenrollment', 'subjectreferral',
        'hivuntested', 'communityengagement', 'pregnancy', 'nonpregnancy', 'stigma', 'stigmaopinion',
        'positiveparticipant', 'substanceuse']
    name_part = 'bcpp_subject_'
    for crf in crf_list:
        crf_table_name = name_part + crf
        crf_sql_statement = f"SELECT * FROM {crf_table_name} INNER JOIN bcpp_subject_subjectvisit ON {crf_table_name}.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
        crf_df = pd.read_sql(crf_sql_statement, con=mysql_cn, columns=['visit_code', 'map_area'])
        crf = crf.title()
        crf_stats = crf_df.groupby('visit_code').size()
        data_dict[crf]=crf_stats.to_dict()
    mysql_cn.close()
    print('===============================Data preparation complete===============================')
    return data_dict

In [30]:
def community_server_frequencies():
    
    # Make a mysql connection
    mysql_cn= MySQLdb.connect(host='127.0.0.1', port=5002,user='root', passwd='cc3721b', db='edc')

    # Visit per survey
    sql_statement = "SELECT * from bcpp_subject_subjectvisit INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_visit_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['visit_code'])    
    visits = subject_visit_df.groupby('visit_code').size()
    
    # Consents
    sql_statement = "SELECT * from bcpp_subject_subjectconsent INNER JOIN member_householdmember ON bcpp_subject_subjectconsent.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_consent_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    consents = subject_consent_df.groupby('map_area').size()
    
    # Household Members
    sql_statement = "SELECT * from member_householdmember INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    household_member_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    members = household_member_df.groupby('map_area').size()
    
    # Requisitions
    sql_statement = "SELECT * FROM bcpp_subject_subjectrequisition INNER JOIN bcpp_subject_subjectvisit ON bcpp_subject_subjectrequisition.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
    subject_requiaitions_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['panel_name', 'visit_code'])
    requisitions = subject_requiaitions_df.groupby(['visit_code', 'panel_name']).size()
    
    data_dict = {
        'Subject Visits': visits.to_dict(),
        'Subject Consents': consents.to_dict(),
        'Household Members': members.to_dict(),
        'Subject requisitions': requisitions.to_dict(),
    }

    crf_list = [
        'demographics', 'education', 'hivtestinghistory', 'hivtestreview', 'residencymobility',
        'hivresultdocumentation', 'hivtested', 'hivcareadherence', 'sexualbehaviour', 'recentpartner',
        'secondpartner', 'thirdpartner', 'hivmedicalcare', 'accesstocare', 'circumcision', 'circumcised',
        'uncircumcised', 'reproductivehealth', 'medicaldiagnoses', 'hypertensioncardiovascular', 'heartattack',
        'cancer', 'hivrelatedillness', 'tuberculosis', 'tbsymptoms', 'qualityoflife', 'resourceutilization',
        'outpatientcare', 'hospitaladmission', 'hivhealthcarecosts', 'labourmarketwages',
        'hivlinkagetocare', 'hivresult', 'elisahivresult', 'pimacd4', 'hicenrollment', 'subjectreferral',
        'hivuntested', 'communityengagement', 'pregnancy', 'nonpregnancy', 'stigma', 'stigmaopinion',
        'positiveparticipant', 'substanceuse']
    name_part = 'bcpp_subject_'
    for crf in crf_list:
        crf_table_name = name_part + crf
        crf_sql_statement = f"SELECT * FROM {crf_table_name} INNER JOIN bcpp_subject_subjectvisit ON {crf_table_name}.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
        crf_df = pd.read_sql(crf_sql_statement, con=mysql_cn, columns=['visit_code', 'map_area'])
        crf = crf.title()
        crf_stats = crf_df.groupby('visit_code').size()
        data_dict[crf]=crf_stats.to_dict()
    mysql_cn.close()
    print('===============================Data preparation complete===============================')
    return data_dict

In [31]:
central_server_data = central_server_frequencies()
community_server_data = community_server_frequencies()

===============================Data preparation complete===============================
===============================Data preparation complete===============================


In [33]:
community_server_data

{'Accesstocare': {'E0': 10, 'T0': 501},
 'Cancer': {'E0': 4, 'T0': 2, 'T1': 1, 'T2': 2},
 'Circumcised': {'E0': 346, 'T0': 58, 'T1': 12, 'T2': 8},
 'Circumcision': {'E0': 808, 'T0': 203, 'T1': 129, 'T2': 92},
 'Communityengagement': {'T0': 503},
 'Demographics': {'E0': 2101, 'T0': 2, 'T1': 40, 'T2': 382},
 'Education': {'E0': 2101, 'T0': 2, 'T1': 40, 'T2': 382},
 'Elisahivresult': {},
 'Heartattack': {'E0': 4, 'T0': 2, 'T1': 1, 'T2': 2},
 'Hicenrollment': {'T0': 1},
 'Hivcareadherence': {'E0': 363, 'T0': 113, 'T1': 118, 'T2': 100},
 'Hivhealthcarecosts': {'T1': 118, 'T2': 101},
 'Hivlinkagetocare': {'T1': 4, 'T2': 24},
 'Hivmedicalcare': {'E0': 364, 'T0': 101, 'T1': 111, 'T2': 101},
 'Hivrelatedillness': {'E0': 2097, 'T0': 2, 'T1': 39, 'T2': 380},
 'Hivresult': {'E0': 1739, 'T0': 396, 'T1': 349, 'T2': 280},
 'Hivresultdocumentation': {'E0': 386, 'T0': 100, 'T1': 1, 'T2': 2},
 'Hivtested': {'E0': 1958, 'T0': 393, 'T1': 90, 'T2': 126},
 'Hivtestinghistory': {'E0': 2101, 'T0': 503, 'T1': 